In [1]:
import sys
import os
import warnings
warnings.filterwarnings("ignore")

os.chdir('/import/home2/yhchenmath/Code/CellSegmentation/')
from torch.utils.data import DataLoader
from utils.scs import dir_to_class
import torch
from options import Options, HParams
from utils.experiman import ExperiMan
from data import find_dataset_using_name

manager = ExperiMan(name='default')
parser = manager.get_basic_arg_parser()
opt = Options(parser).parse()  # get training options

manager.setup(opt)
opt = HParams(**vars(opt))

2023-07-30 20:19:01.504262: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-30 20:19:02.662285: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


======> UID of this run: 230730-201904-rTc2Gk
run_number : 0
Directory /import/home2/yhchenmath/Log/CellSeg/SCS/UNDEFINED/0 exists, please choose an option:
Old files deleted.
======> Directory for this run: /import/home2/yhchenmath/Log/CellSeg/SCS/UNDEFINED/0
2023-07-30 20:20:00,118 INFO Opts: Namespace(dataset={'name': 'scs_mouse_brain', 'loc': 'SCS', 'patch_size': 1200, 'bin_size': 3, 'n_neighbor': 50, 'max_patch_num': 1}, model={'name': 'scs', 'use_amp': False, 'use_sam': False, 'arch': {'class_num': 16, 'input_shape': 2000, 'input_position_shape': 2, 'projection_dim': 64, 'num_heads': 1, 'transformer_units': 128, 'transformer_layers': 8, 'mlp_head_units': [1024, 256]}, 'optimizer': {'name': 'adamw', 'lr': 0.001, 'weight_decay': 0.0001}, 'schedulers': None}, training={'num_epochs': 1, 'batch_size': 10}, gpu='0', code_dir='./', data_dir='/import/home2/yhchenmath/Dataset/CellSeg', log_dir='/import/home2/yhchenmath/Log/CellSeg', exp_name='SCS', run_name='UNDEFINED', run_number='0', se

In [2]:
assert torch.cuda.is_available(), "CPU training is not allowed."
logger = manager.get_logger()
logger.info(f"======> Single GPU Training")
# Set up tensorboard
manager._third_party_tools = ('tensorboard',)
manager._setup_third_party_tools()

# Create dataset
dataset_all_patches = find_dataset_using_name(dataset_name=opt.dataset.name)(opt, manager)
logger.info("======> dataset [%s] was created" % type(dataset_all_patches).__name__)
logger.info(f"======> length of dataset: {len(dataset_all_patches)}")

# Method
logger.info(f"======> Model: "+ opt.model.name)
if opt.model.name == "scs":
    from models.scs_model import SCSModel as Model
else:
    raise NotImplementedError

logger.info(f"======> creating model")
model = Model(
    opt = opt,
    manager = manager,
    dataloader = None
).cuda()

2023-07-30 20:20:00,411 INFO ======> Single GPU Training
|-----> Constructing count matrices.
|-----> <insert> __type to uns in AnnData Object.
|-----> <insert> pp to uns in AnnData Object.
|-----> <insert> spatial to uns in AnnData Object.
2023-07-30 20:21:29,657 INFO ======> 108 patches will be processed.
|-----> <select> stain layer in AnnData Object
|-----> <select> unspliced layer in AnnData Object
|-----> Refining alignment in rigid mode.


Loss -1.5378e-02: 100%|████████████████████████████████████████████████████████████████| 100/100 [00:06<00:00, 15.49it/s]

|-----> Transforming layers ['stain']
|-----> <select> stain layer in AnnData Object
|-----> <insert> stain to layers in AnnData Object.
|-----> <select> stain layer in AnnData Object
|-----> Constructing nuclei mask from staining image.


|-----> <insert> stain_mask to layers in AnnData Object.
|-----> <select> stain_mask layer in AnnData Object
|-----> Finding peaks with minimum distance 7.
|-----> <insert> stain_distances to layers in AnnData Object.
|-----> <insert> stain_markers to layers in AnnData Object.
|-----> <select> stain layer in AnnData Object
|-----> <select> stain_mask layer in AnnData Object
|-----> <select> stain_markers layer in AnnData Object
|-----> Running Watershed.
|-----> <insert> watershed_labels to layers in AnnData Object.
2023-07-30 20:26:27,761 INFO ======> dataset [SCSMouseBrainDataset] was created
2023-07-30 20:26:27,763 INFO ======> length of dataset: 1
2023-07-30 20:26:27,764 INFO ======> Model: scs
2023-07-30 20:26:27,790 INFO ======> creating model


In [3]:
per_patch_dataset = dataset_all_patches[0]

In [4]:
per_patch_loader = DataLoader(
    per_patch_dataset,
    batch_size = opt.training.batch_size,
    num_workers = 1,
    shuffle=True
)

for x_train, x_train_pos, y_train, y_binary_train in per_patch_loader:
    break

In [5]:
x_train.shape, x_train_pos.shape, y_train.shape, y_binary_train.shape

(torch.Size([10, 50, 2000]),
 torch.Size([10, 50, 2]),
 torch.Size([10, 2]),
 torch.Size([10]))

In [6]:
x_train_pos_ =x_train_pos
for i in range(len(x_train_pos_)):
    for j in range(1, len(x_train_pos_[i])):
        x_train_pos_[i][j] = x_train_pos_[i][j] - x_train_pos_[i][0]
    x_train_pos_[i][0] = x_train_pos_[i][0] - x_train_pos_[i][0]

In [7]:
for i in range(len(y_train)):
    if y_train[i][0] != -1:
        y_train[i] = y_train[i] - x_train_pos[i][0]
    else:
        y_train[i][0] = -9999
        y_train[i][1] = -9999
y_train = dir_to_class(y_train, opt.model.arch.class_num)
y_train = torch.from_numpy(y_train)

In [8]:
y_train.shape

torch.Size([10, 16])

In [9]:
total_num_steps = 0
losses, predictions = model.train_step({
    'x': [x_train.float(), x_train_pos.float()],
    'y': [y_train, y_binary_train.unsqueeze(1).float()]
})
total_num_steps += 1
logger.info(f"iter {total_num_steps} | {losses}")

2023-07-30 20:26:30,893 INFO iter 1 | {'loss.global': 2.0916177463531493, 'loss.pos': 1.391213400363922, 'loss.bi': 0.7004043459892273}
